# Generating Essays Using a Step-by-Step Approach With LangChain

<a target="_blank" href="https://colab.research.google.com/github/koverholt/gemini-agent-ai-camp/blob/main/2-langchain-essay/langchain-essay.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This notebook demonstrates how to build an essay-writing pipeline using [LangChain](https://www.langchain.com/), the [Gemini API in Google AI Studio](https://ai.google.dev/gemini-api/docs), and [Tavily](https://tavily.com/) for search.

By combining these tools, we create a seamless workflow that plans an essay outline, performs web searches for relevant information, and generates a complete essay draft based on the collected data.

This solution showcases the power of chaining LLM models and external tools to tackle complex tasks with minimal human intervention, providing a robust approach to automated content generation.

<img src="../images/2-langchain-essay.png" width="550px">

## Install depdencies

In [1]:
!pip install -q -U \
    langchain \
    langchain-google-genai \
    langchain-community \
    tavily-python

## Import packages

In [2]:
import google.generativeai as genai
from IPython.display import Image, display, Markdown
from langchain import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser

## Configure API keys

Get API keys from [Google AI Studio](https://ai.google.dev/gemini-api/docs/api-key) and [Tavily](https://tavily.com/).

In [3]:
genai.configure(api_key="YOUR_GOOGLE_AI_STUDIO_API_KEY")
os.environ["TAVILY_API_KEY"] = "YOUR_TAVILY_API_KEY"

## Initialize Gemini model

In [4]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002", temperature=0.7)

## Initialize search tool

In [5]:
tavily_tool = TavilySearchResults(max_results=5)

## Define prompt templates and Runnables

In [6]:
# Planning: Create an outline for the essay
outline_template = ChatPromptTemplate.from_template(
    "Create a detailed outline for an essay on {topic}"
)


# Research: Web search
def research_fn(topic):
    response = tavily_tool.invoke({"query": topic})
    return "\n".join([f"- {result['content']}" for result in response])


# Writing: Write the essay based on outline and research
writing_template = ChatPromptTemplate.from_template(
    "Based on the following outline and research, write a 3-paragraph essay on '{topic}':\n\nOutline:\n{outline}\n\nResearch:\n{research}\n\nEssay:"
)

## Define the Runnable Chain using LCEL

In [7]:
# Define individual chains
outline_chain = LLMChain(llm=model, prompt=outline_template)
writing_chain = LLMChain(llm=model, prompt=writing_template)

# Use the pipe operator to combine chains
chain = (
    outline_chain
    | (
        lambda result: {
            "topic": result["topic"],
            "outline": result["text"],
            "research": research_fn(result["topic"]),
        }
    )
    | writing_chain
    | (lambda result: result["text"])  # Extract the essay text from the final result
    | StrOutputParser()
)

/var/folders/lb/v1hjvf8n7yd98mxj_t_h55t80000gn/T/ipykernel_90023/3169385258.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  outline_chain = LLMChain(llm=model, prompt=outline_template)


## Generate an essay on a given topic

In [8]:
topic = "Write an essay about the impacts of Hurricane Helene in 2024"
essay = chain.invoke({"topic": topic})
display(Markdown(essay))

The increasing frequency and intensity of hurricanes are stark reminders of the changing climate.  In 2024, the hypothetical Hurricane Helene, a powerful Category 4 storm, ripped through the Atlantic, leaving a trail of destruction in its wake. Though its direct landfall spared major population centers in Florida, Helene's widespread impacts underscored the interconnectedness of ecosystems and the vulnerability of coastal communities to even indirect hurricane effects. From severe flooding in North Carolina to coastal erosion in Florida, Helene's fury served as a potent example of the multifaceted consequences of these extreme weather events.

Environmentally, Hurricane Helene wreaked havoc.  Coastal areas in Florida experienced significant beach erosion and dune damage due to the storm surge and powerful waves, jeopardizing crucial ecosystems and coastal infrastructure. Inland, particularly in North Carolina, heavy rainfall led to record flooding, contaminating water supplies, destroying wildlife habitats, and inundating agricultural lands.  The storm's impact on marine life was also substantial, potentially disrupting coral reefs, fish populations, and sea turtle nesting sites.  Further compounding the environmental damage was the massive amount of debris – plastics, building materials, and vegetation – scattered across affected areas, posing a long-term threat to the environment.

Economically, the costs of Hurricane Helene were staggering.  The storm inflicted extensive damage to infrastructure, crippling roads, bridges, power grids, and communication networks. Businesses faced significant economic losses from closures, property damage, and supply chain disruptions, particularly those in the tourism and related industries. The financial burden on governments and aid organizations for search and rescue, emergency shelter, medical care, and rebuilding efforts was immense.  Furthermore, the sheer volume of insurance claims for property damage and business interruption placed a significant strain on the insurance industry.  The long-term economic recovery from Helene promised to be a slow and arduous process.
